***GENERATED CODE FOR classificationhealthcare PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AdmissionDate', 'transformation_label': 'String Indexer'}], 'feature': 'AdmissionDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '1/1/2023', 'max': '9/9/2022', 'missing': '0', 'distinct': '300'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AdmissionDate'}, {'feature_label': 'AdmissionDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AdmissionDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Hospital', 'transformation_label': 'String Indexer'}], 'feature': 'Hospital', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'City Hospital', 'max': 'Regional Medical Center', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Hospital'}, {'feature_label': 'Hospital', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Hospital')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AdmissionType', 'transformation_label': 'String Indexer'}], 'feature': 'AdmissionType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Elective', 'max': 'Urgent', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AdmissionType'}, {'feature_label': 'AdmissionType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AdmissionType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InsuranceType', 'transformation_label': 'String Indexer'}], 'feature': 'InsuranceType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Medicaid', 'max': 'Self-pay', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InsuranceType'}, {'feature_label': 'InsuranceType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InsuranceType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Treatment', 'transformation_label': 'String Indexer'}], 'feature': 'Treatment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Consultation', 'max': 'Therapy', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Treatment'}, {'feature_label': 'Treatment', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Treatment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Doctor', 'transformation_label': 'String Indexer'}], 'feature': 'Doctor', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Dr. Brown', 'max': 'Dr. Williams', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Doctor'}, {'feature_label': 'Doctor', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Doctor')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TreatmentCost', 'threshold': 10160.58, 'transformation_label': 'Binarizer'}], 'feature': 'TreatmentCost', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '10161.1', 'stddev': '5627.71', 'min': '562.76', 'max': '19967.78', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'TreatmentCost'}, {'feature_label': 'TreatmentCost', 'threshold': 10160.58, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('TreatmentCost')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Outcome', 'transformation_label': 'String Indexer'}], 'feature': 'Outcome', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Deteriorated', 'max': 'Unchanged', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Outcome'}, {'feature_label': 'Outcome', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Outcome')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run classificationhealthcareHooks.ipynb
try:
	#sourcePreExecutionHook()

	enhancedhealthcaredata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Enhanced_Healthcare_Data.csv', 'filename': 'Enhanced_Healthcare_Data.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(enhancedhealthcaredata)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run classificationhealthcareHooks.ipynb
try:
	#transformationPreExecutionHook()

	classificationhealthcareautofe = TransformationMain.run(enhancedhealthcaredata,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "AdmissionDate", "transformation_label": "String Indexer"}], "feature": "AdmissionDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/1/2023", "max": "9/9/2022", "missing": "0", "distinct": "300"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AdmissionDate"}, {"transformationsData": [{"feature_label": "Hospital", "transformation_label": "String Indexer"}], "feature": "Hospital", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "City Hospital", "max": "Regional Medical Center", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Hospital"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PatientID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "10249.5", "stddev": "144.48", "min": "10000", "max": "10499", "missing": "0"}, "updatedLabel": "PatientID"}, {"transformationsData": [{"feature_label": "AdmissionType", "transformation_label": "String Indexer"}], "feature": "AdmissionType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Elective", "max": "Urgent", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AdmissionType"}, {"transformationsData": [{"feature_label": "InsuranceType", "transformation_label": "String Indexer"}], "feature": "InsuranceType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Medicaid", "max": "Self-pay", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InsuranceType"}, {"transformationsData": [{"feature_label": "Treatment", "transformation_label": "String Indexer"}], "feature": "Treatment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Consultation", "max": "Therapy", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Treatment"}, {"transformationsData": [{"feature_label": "Doctor", "transformation_label": "String Indexer"}], "feature": "Doctor", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Dr. Brown", "max": "Dr. Williams", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Doctor"}, {"transformationsData": [{"feature_label": "TreatmentCost", "threshold": 10160.58, "transformation_label": "Binarizer"}], "feature": "TreatmentCost", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "10161.1", "stddev": "5627.71", "min": "562.76", "max": "19967.78", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "TreatmentCost"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LengthOfStay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "7.6", "stddev": "4.1", "min": "1", "max": "14", "missing": "0"}, "updatedLabel": "LengthOfStay"}, {"transformationsData": [{"feature_label": "Outcome", "transformation_label": "String Indexer"}], "feature": "Outcome", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Deteriorated", "max": "Unchanged", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Outcome"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Churn_Flag", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.52", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Churn_Flag"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2022.27", "stddev": "0.44", "min": "2022", "max": "2023", "missing": "0"}, "updatedLabel": "Year"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "5.61", "stddev": "3.4", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "Month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Quarter", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.23", "stddev": "1.09", "min": "1", "max": "4", "missing": "0"}, "updatedLabel": "Quarter"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "YoY_TreatmentCost", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "498", "mean": "95.74", "stddev": "318.39", "min": "-95.76452453", "max": "3031.086076", "missing": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "YoY_TreatmentCost"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MoM_TreatmentCost", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "483", "mean": "96.58", "stddev": "321.6", "min": "-95.76452453", "max": "3031.086076", "missing": "17"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MoM_TreatmentCost"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "QoQ_TreatmentCost", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "494", "mean": "95.41", "stddev": "318.48", "min": "-95.76452453", "max": "3031.086076", "missing": "6"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "QoQ_TreatmentCost"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "YoY_LengthOfStay", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "498", "mean": "79.27", "stddev": "249.59", "min": "-92.85714286", "max": "1300.0", "missing": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "YoY_LengthOfStay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MoM_LengthOfStay", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "483", "mean": "79.8", "stddev": "250.19", "min": "-92.85714286", "max": "1300.0", "missing": "17"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MoM_LengthOfStay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "QoQ_LengthOfStay", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "494", "mean": "80.09", "stddev": "250.42", "min": "-92.85714286", "max": "1300.0", "missing": "6"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "QoQ_LengthOfStay"}]}))

	#transformationPostExecutionHook(classificationhealthcareautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run classificationhealthcareHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(classificationhealthcareautofe, ["PatientID", "LengthOfStay", "Year", "Month", "Quarter", "YoY_TreatmentCost", "MoM_TreatmentCost", "QoQ_TreatmentCost", "YoY_LengthOfStay", "MoM_LengthOfStay", "QoQ_LengthOfStay", "AdmissionDate_stringindexer", "Hospital_stringindexer", "AdmissionType_stringindexer", "InsuranceType_stringindexer", "Treatment_stringindexer", "Doctor_stringindexer", "TreatmentCost_binarizer", "Outcome_stringindexer"], "Churn_Flag")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

